In [42]:
import pickle
with open('gaussian_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [43]:
def mol_to_xyz(mol, file_path="mol.xyz"):
    num_atoms = mol.GetNumAtoms()
    xyz_content = f"{num_atoms}\n\n"
    conf = mol.GetConformer()
    for atom in mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        xyz_content += f"{atom.GetSymbol()} {pos.x:.6f} {pos.y:.6f} {pos.z:.6f}\n"
    with open(file_path, 'w+') as f:
        f.write(xyz_content)

In [44]:
import openbabel
from rdkit import Chem
from rdkit.Chem import AllChem

def xyz_to_sdf_to_mol(xyz_file, sdf_file="temp.sdf"):
    

    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("xyz", "sdf")
    mol = openbabel.OBMol()
    if not obConversion.ReadFile(mol, xyz_file):
        raise FileNotFoundError(f"Unable read xyz: {xyz_file}")

    obConversion.WriteFile(mol, sdf_file)
    rdkit_mol = Chem.SDMolSupplier(sdf_file)[0]
    if rdkit_mol is None:
        raise ValueError("convertion error")

    return rdkit_mol

def gjf_to_sdf_to_mol(gjf_file, sdf_file="temp.sdf"):
    

    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("gjf", "sdf")
    mol = openbabel.OBMol()
    if not obConversion.ReadFile(mol, gjf_file):
        raise FileNotFoundError(f"Unable read gjf: {gjf_file}")

    obConversion.WriteFile(mol, sdf_file)
    rdkit_mol = Chem.SDMolSupplier(sdf_file)[0]
    if rdkit_mol is None:
        raise ValueError("convertion error")

    return rdkit_mol

def get_adjacent_atoms(mol, atom_index):
    atom = mol.GetAtomWithIdx(atom_index)  # 获取指定索引的原子
    adjacent_atoms = [nbr.GetIdx() for nbr in atom.GetNeighbors()]  # 获取所有邻接原子的索引
    return adjacent_atoms

In [45]:
def draw_xyz(xyz_path):
    mol = xyz_to_sdf_to_mol(xyz_path)
    from rdkit import Chem
    from rdkit.Chem.Draw import rdMolDraw2D
    from IPython.display import SVG
    d2d = rdMolDraw2D.MolDraw2DSVG(350,300)
    d2d.drawOptions().addAtomIndices=True
    d2d.DrawMolecule(mol)
    d2d.FinishDrawing()
    SVG(d2d.GetDrawingText())
    
    return mol

def draw_gjf(xyz_path):
    mol = gjf_to_sdf_to_mol(xyz_path)
    from rdkit import Chem
    from rdkit.Chem.Draw import rdMolDraw2D
    from IPython.display import SVG
    d2d = rdMolDraw2D.MolDraw2DSVG(350,300)
    d2d.drawOptions().addAtomIndices=True
    d2d.DrawMolecule(mol)
    d2d.FinishDrawing()
    SVG(d2d.GetDrawingText())
    
    return mol

In [51]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem

def mol_to_gjf(mol, atom1, atom2, gjf_path, nproc=64, memory=512, chk="temp.chk"):
    
    if atom1 > atom2:
        atom1, atom2 = atom2, atom1
    
    xyz_filename = "__temp__.xyz"
    
    mol.UpdatePropertyCache(strict=True)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, randomSeed=42)
    AllChem.UFFOptimizeMolecule(mol)

    # 创建XYZ文件
    mol_to_xyz(mol, "__temp__.xyz")
    
    # 使用Open Babel将XYZ转换为GJF
    temp_gjf_filename = "__temp__.gjf"
    temp_sdf_filename = "__temp__.sdf"
    
    command = f"obabel -ixyz {xyz_filename} -o gjf -xk \"#p M062X def2svp opt=modredundant nosymm\" > {temp_gjf_filename}"
    os.system(command)
    
    command = f"obabel -ixyz {xyz_filename} -o sdf > {temp_sdf_filename}"
    os.system(command)
    
    mol_with_bond = Chem.SDMolSupplier(temp_sdf_filename)[0]
    
    atom1_adjs = get_adjacent_atoms(mol_with_bond, atom1)
    atom1_adjs.remove(atom2)
    
    atom2_adjs = get_adjacent_atoms(mol_with_bond, atom2)
    atom2_adjs.remove(atom1)
    
    assert len(atom1_adjs) >=1 and len(atom2_adjs) >= 1, "atom1 and atom2 lay in the terminal of the molecule"
    
    atom1_adj = atom1_adjs[0]
    atom2_adj = atom2_adjs[0]
    
    # 读取转换后的GJF文件
    with open(temp_gjf_filename, "r") as file:
        gjf_lines = file.readlines()
    
    # 准备最终的GJF文件
    final_gjf_filename = gjf_path
    with open(final_gjf_filename, "w") as file:
        file.write(f"%nprocshared={nproc}\n")
        file.write(f"%mem={memory}GB\n")
        file.write(f"%chk={chk}\n")
        file.write(gjf_lines[0] + "\n")
        file.write("Title Card Required\n\n")
        for line in gjf_lines[4:]:
            file.write(line)
        
        # 添加柔性扫描命令
        file.write(f"{atom1_adj} {atom1} {atom2} {atom2_adj} S 36 10.0\n")
        file.write("\n"*2)
    
    # 清理临时文件
    os.remove(xyz_filename)
    os.remove(temp_sdf_filename)
    os.remove(temp_gjf_filename)
    
    return mol_with_bond

In [ ]:
for index, mol in enumerate(data["mol"]):
    gjf_path = os.path.join("gaussian_3k", f"{index}.gjf")
    atom1, atom2 = data["axis_index"][index]
    try:
        _ = mol_to_gjf(mol, atom1, atom2, gjf_path, nproc=64, memory=512, chk=f"temp_{index}.chk")
    except:
        continue
    